In [1]:
import pandas as pd
import numpy as np
%matplotlib inline
from IPython.display import HTML

import dash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output
from jupyter_dash import JupyterDash

import plotly
import plotly.graph_objs as go
import plotly.express as px

plotly.io.renderers.default = 'notebook'

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
The raw code is hidden for easier reading. To toggle on/off the raw code, click <a href="javascript:code_toggle()">here</a>.''')

In [2]:
#reading data
print('Reading data ...')

data_dir = '/home/andry/projects/dataKind/DC-DHS-Public/data/'
data0 = pd.read_csv(data_dir+"InShelterPerDay_Shelter_10-06-2021.csv")
data = pd.read_csv(data_dir+"InShelterPerDay_Gender_Shelter_10-06-2021.csv")

data0['date'] = pd.to_datetime(data0['date'])
data['date'] = pd.to_datetime(data['date'])
data['total'] = data[['Male', 'Female', 'NA', 'Transgender']].sum(axis=1)
# data = data[data['Total']!=0]

Reading data ...


In [3]:
# %%javascript
# IPython.OutputArea.prototype._should_scroll = function(lines) {
#    return false;
# }

unique_name = data['name'].unique()
unique_name.sort()
name_dict = {}
for i,name in enumerate(unique_name):
    name_dict[name] = i 

data['name_id'] = data['name'].map(name_dict)
data0['name_id'] = data0['name'].map(name_dict)

In [4]:

app = JupyterDash(__name__)

min_date = data['date'].dt.year.min()
max_date = data['date'].dt.year.max()


graph_style = {'display': 'inline-block', 'width': '80%','text-align': 'center', 'vertical-align': 'middle'}
menu_style = {'display': 'inline-block', 'width': '100%','text-align': 'left', 'vertical-align': 'left'} 
div0_style = {'display': 'inline-block', 'width': '80%','text-align': 'center', 'vertical-align': 'middle'}



app.layout = html.Div([
        # menu: selection 
        html.Div([
            html.H1("Beds Used by Shelter and Gender"),
            dcc.Dropdown(
                id='colorscale-dropdown', clearable=False,
                value=['All_Shelter'], options=[{'label': 'All Shelter', 'value': 'All_Shelter'}] + [
                    {'label': c, 'value': c}
                    for c in np.sort(unique_name)],
                # placeholder=['All_Shelter'],
                multi=True,
                style={'width': '80%','color': '#212121',}
                ),
            dcc.Checklist(
                id="gender",
                options=[{'label': c, 'value': c} for c in ['All Gender', 'Male', 'Female']],
                value=['All Gender', 'Male', 'Female'],
                labelStyle={"display": "inline-block", 'marginTop': '15px', 'marginRight': '15px'},
                style={'width': '60%','color': '#212121',}
                )
            ], style=menu_style),

        # graph and slider
        html.Div([
            dcc.Graph(id='graph'),
            dcc.RangeSlider(
                id='date-range-slider',
                min=min_date, max=max_date,
                step=None,
                value=[min_date, max_date],
                marks={str(year): str(year) for year in range(min_date, max_date+1)},
                ),
            ],
            style=graph_style),
        ],
        style=div0_style)
    


# Define callback to update graph
@app.callback(
    Output('graph', 'figure'),
    [Input("colorscale-dropdown", "value"),
    Input('gender', 'value'),
    Input('date-range-slider', 'value')]
)
def update_figure(shelter, genders, date_range):
    df = data[(data['date'] >= str(date_range[0])) & (data['date']<= str(date_range[1]+1))]
    if len(shelter) == 0:
        df = df.groupby('date').sum().reset_index()
    elif (shelter[0] == 'All_Shelter') and len(shelter) == 1:
        df = df.groupby('date').sum().reset_index()
    else:
        df = df[df['name'].isin(shelter)].groupby('date').sum().reset_index()
    fig = go.Figure()
    for gender in genders:
        if gender == 'All Gender': 
            fig.add_scatter(x=df['date'], y = df[['Male', 'Female', 'NA', 'Transgender']].sum(axis=1), mode = 'lines', name=gender)
        else:
            fig.add_scatter(x=df['date'], y = df[gender], mode = 'lines', name=gender)
    fig.update_layout(xaxis_title='Date', margin={'l': 100, 'b': 40, 't': 20, 'r': 20}, 
                      yaxis_title='Shelter beds used', hovermode="x unified")
    return fig



app.run_server(mode='inline')

In [5]:

app1 = JupyterDash(__name__)

min_date = data['date'].dt.year.min()
max_date = data['date'].dt.year.max()


graph_style = {'display': 'inline-block', 'width': '80%','text-align': 'center', 'vertical-align': 'middle'}
menu_style = {'display': 'inline-block', 'width': '100%','text-align': 'left', 'vertical-align': 'left'} 
div0_style = {'display': 'inline-block', 'width': '80%','text-align': 'center', 'vertical-align': 'middle'}


app1.layout = html.Div([
        html.Div([
            html.H1("Individual Shelter: Beds used"),
            html.P(id='hover-data')],
            style=menu_style),
        html.Div([
            html.Div([dcc.Graph(id='graph_shelter')],
                    style={'width': '90%'}),
            
            dcc.RangeSlider(
                id='date-range-slider_shelter',
                min=min_date, max=max_date,
                step=None,
                value=[min_date, max_date],
                marks={str(year): str(year) for year in range(min_date, max_date+1)},
                ),
            ],
            style=graph_style),
        ],
        style=div0_style)


# Define callback to update graph
@app1.callback(
    Output('graph_shelter', 'figure'),
    [Input('date-range-slider_shelter', 'value')]
)
def update_figure_shelter(date_range):
    # data1 = data.groupby(['name_id','date']).sum().reset_index(level='date')
    df0 = data[(data['date'] >= str(date_range[0])) & (data['date']<= str(date_range[1]+1))]
    fig = go.Figure(layout=dict(hovermode= "closest", showlegend=False, margin={'l': 100, 'b': 40, 't': 20, 'r': 20}, 
                      xaxis_title='Date', yaxis_title='Shelter beds used'))

    df = df0.copy() 
    for c in df.groupby(['name_id'])['total'].mean().sort_values(ascending=False)[:10].index:
        df2 = df0[df0.name_id == c]
        fig.add_scattergl(x=df2['date'], y = df2['total'], mode = 'lines', name=None, customdata=[c] * len(df2))
    
    return fig


@app1.callback(
    Output('hover-data', 'children'),
    Input('graph_shelter', 'hoverData'))
def display_hover_data(hoverData):
    if hoverData is None:
        return 'Hovered: None'
    return 'Hovered: '+unique_name[hoverData['points'][0]['customdata']]


app1.run_server(mode='inline', port=8060)